In [1]:
import urllib.request
import os

import cv2
import numpy as np

In [2]:
#  modify url_base to get different subVideo from whole video
url_base = 'http://hls.cntv.myalicdn.com/asp/hls/1200/0303000a/3/default/5af52768f77e4bbc9f3fb5c3e8027c8e/'
# 20180104
# http://hls.cntv.myalicdn.com/asp/hls/1200/0303000a/3/default/5af52768f77e4bbc9f3fb5c3e8027c8e/0.ts
outputVideoFileName = 'completed_video.avi'

In [3]:
count = 0
while(True):
    url_link = url_base + str(count) + '.ts'
    try:
        urllib.request.urlretrieve(url_link,str(count)+'.ts')
        count += 1
    except Exception as e:
        break

In [4]:
print(count)

180


In [5]:
#  get them together

subVideo_startIndex = 0
subVideo_endIndex = count  # 180
subVideo_format = '.ts'
#  将需要的ts子视频加入到列表中
videoList = []  # 列表类型长度 => len(list)
for i in range(subVideo_startIndex,subVideo_endIndex,1):  
    videoList.append(cv2.VideoCapture(str(i)+subVideo_format))

#  获取视频的基本信息。FPS，每一帧的高宽
fps = videoList[0].get(cv2.CAP_PROP_FPS)
size = int(videoList[0].get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoList[0].get(cv2.CAP_PROP_FRAME_HEIGHT))  # 不可以是tuple类型

#  通过基本信息构建视频写入对象
videoWriter = cv2.VideoWriter(outputVideoFileName, cv2.VideoWriter_fourcc('M','P','4','2'),fps,size)

#  循环遍历每一个子视频的frame
#  ATTENTION! 每个子视频最后的frame是空的，作为EOF存在，但是success=True。所以要用空frame来判断当前视频是否结束
for i in range(len(videoList)):
    success, frame = videoList[i].read()
    while (not not np.shape(frame)):  # not empty_list = True, 当列表为空时，true。多一个not，列表为空时F
        videoWriter.write(frame)  # 写入所有notEOF-frame
        success, frame = videoList[i].read()  # 读取当前视频的下一个Frame
        
#  不含如EOF-frame